In [13]:
% setup environment
% set path to equilibrium solver
txt=pwd; n=length(txt); rootpath=txt;
last_slash_pos = rindex(rootpath, "/");
rootpath=rootpath(1:last_slash_pos);
JEQUILIBpath=[rootpath,'/JEQUILIB'];
addpath(JEQUILIBpath);
% clear data and number format
clear; format short e
% turn off warnings
warning off
%graphics_toolkit ("notebook"); 
graphics_toolkit ("plotly"); 
%graphics_toolkit("gnuplot")

In [14]:
function [Ag,Cl,AgCls,MASSERR]=AgCltableaumorecomplex(pH,pe,T,flag1,flag2,flag3,flag4,flag5)

% input tableau.  change this part % ----------------------------------------------

Tableau=[...
%H      e        Ag        Cl     logK                                phase    species1 
1       0        0         0      0                                   0    {'H'}
0       1        0         0      0                                   0    {'e'}
0       0        1         0      0                                   0    {'Ag'}
0       0        0         1      0                                   0    {'Cl'}
-1      0        0         0      -14                                 0    {'OH'}
0       0        1         1      3.2971                              0    {'AgCl'}
0       0        1         2      5.2989                              0    {'AgCl2'}
0       0        1         3      5.1310                              0    {'AgCl3'}
0       0        1         4      3.8050                              0    {'AgCl4'} 
%solids
0       0        1         1      9.7453                              1    {'AgCls'}
];

% end of tableau.  ------------------ % ----------------------------------------------

[KSOLID,ASOLID,SOLIDNAMES,KSOLUTION,ASOLUTION,SOLUTIONNAMES]=processtableau(Tableau,pH,pe);

[SPECIESCONCS,SPECIATIONNAMES,MASSERR,X]=returnspeciationRE(KSOLID,ASOLID,SOLIDNAMES,KSOLUTION,ASOLUTION,SOLUTIONNAMES,T,flag1,flag2,flag3,flag4,flag5);

for k=1:size(SPECIESCONCS,1)
      txt=[SPECIATIONNAMES(k,:),'=SPECIESCONCS(k);'];
      eval(txt)
end

end


In [15]:
% MODEL USING TABLEAU (different gradients or log versus linear variables)
%graphics_toolkit("notebook")
%model with just inorganic speciation
%geochemical model
% input the constants and solution chemistry

flag1=2; %flag1 for concentration based search vector (1) or for log based search vector (2).
flag2=1; %flag2 for numerical derivatives (2) or analtyical derivatives (1)
database=[]; 
flag3=0; % flag3 1 for show warnings. 0 no warnings
flag4=0; %if=1 solve tableau one line at a time to get a good initial guess (no solids). really challenging probs only
flag5=0; %if=1 use stored initial guess for including solids

AgT=1e-4; ClT=1e-5; pH=7; 

inorganicTOTALS=[AgT ClT];
inorganicTOTALS(inorganicTOTALS==0)=1e-16; % get rid of zero values, b/c div by zero error
TOTALS=[inorganicTOTALS]; %TOTALS=inorganicTOTALS;
pH=pH; pe=20.75-pH; 
tic;
[Ag,Cl,AgCls,MASSERR]=AgCltableaumorecomplex(pH,pe,TOTALS',flag1,flag2,flag3,flag4,flag5);
tableautime=toc;
Agmasserror=MASSERR(1); Clmasserror=MASSERR(2);

In [16]:
% exact model

Ksp=10^-9.7453;

% check SI

IAP=AgT*ClT;

Agexact=AgT; Clexact=ClT; AgClsexact=0;

if IAP>=Ksp
a=1; b=ClT-AgT; c=-Ksp;
t=roots([a b c]); t=t(t>0); t=real(t(imag(t)==0));
Agexact=t; AgClsexact=AgT-Ag; Clexact=ClT-AgClsexact;
end


In [17]:
% compare models

Agcompare=[Ag Agexact]
Clcompare=[Cl Clexact]
AgClcompare=[AgCls AgClsexact]
Agmasserrorcompare=[Agmasserror ]
Clmasserrorcompare=[Clmasserror ]
timecompare=[tableautime]

Agcompare,1,2
1,9.1955e-05,9.19549e-05


Clcompare,1,2
1,1.9549e-06,1.95497e-06


AgClcompare,1,2
1,7.68867e-06,8.04503e-06


Agmasserrorcompare = 3.0791e-17


Clmasserrorcompare = 6.5391e-19


timecompare = 3.7611e-02
